In [1]:
# !pip install datasets

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
dataset = load_dataset('imdb')

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
train_texts = dataset['train']['text']
corpus = "".join(train_texts)

print("corpus length:", len(corpus))
print(corpus[:500])

corpus length: 33126741
I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attent


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# sort the corpus and collect only unique characters

chars = sorted(list(set(corpus)))
vocab_size = len(chars)

print(f"unique characters{chars[:50]}")
print(f"Vocabulary size {vocab_size}")

# Map character with index

char_to_idx = {c:i for i,c in enumerate(chars)}
idx_to_char = {i:c for i,c in enumerate(chars)}

print(char_to_idx)

encoded = np.array([char_to_idx[c] for c in corpus])

print("Encoded Sample", encoded[:100])

unique characters['\x08', '\t', '\x10', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']
Vocabulary size 178
{'\x08': 0, '\t': 1, '\x10': 2, ' ': 3, '!': 4, '"': 5, '#': 6, '$': 7, '%': 8, '&': 9, "'": 10, '(': 11, ')': 12, '*': 13, '+': 14, ',': 15, '-': 16, '.': 17, '/': 18, '0': 19, '1': 20, '2': 21, '3': 22, '4': 23, '5': 24, '6': 25, '7': 26, '8': 27, '9': 28, ':': 29, ';': 30, '<': 31, '=': 32, '>': 33, '?': 34, '@': 35, 'A': 36, 'B': 37, 'C': 38, 'D': 39, 'E': 40, 'F': 41, 'G': 42, 'H': 43, 'I': 44, 'J': 45, 'K': 46, 'L': 47, 'M': 48, 'N': 49, 'O': 50, 'P': 51, 'Q': 52, 'R': 53, 'S': 54, 'T': 55, 'U': 56, 'V': 57, 'W': 58, 'X': 59, 'Y': 60, 'Z': 61, '[': 62, '\\': 63, ']': 64, '^': 65, '_': 66, '`': 67, 'a': 68, 'b': 69, 'c': 70, 'd': 71, 'e': 72, 'f': 73, 'g': 74, 'h': 75, 'i': 76, 'j': 77, 'k':

In [7]:
import tensorflow as tf

# Length of each sequence (window size)
# 100 characters per training
seq_length = 100

# Create dataset from encoded numbers
char_dataset = tf.data.Dataset.from_tensor_slices(encoded)

# Create windows of length (seq_length + 1)
# +1 because we need input sequence and target (shifted by 1)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Function to split into input and target
def split_input_target(chunk):
    input_text = chunk[:-1]   # first 100 characters
    target_text = chunk[1:]   # next 100 characters
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Shuffle and batch
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)


<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [8]:
# Vocabulary size
vocab_size = len(chars)

# Embedding dimension (size of vector for each character)
embedding_dim = 256

# Number of LSTM units
rnn_units = 512

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=100),  # 100 = seq_length
    tf.keras.layers.LSTM(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])

# Define the loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer='adam', loss=loss)


for input_example, target_example in dataset.take(1):
    example_batch_predictions = model(input_example)


model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 100, 256)         │        45,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (64, 100, 512)         │     1,574,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 100, 178)         │        91,314 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,711,794 (6.53 MB)

 Trainable params: 1,711,794 (6.53 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
small_dataset = dataset.take(200)   # only first 200 batches
history = model.fit(small_dataset, epochs=5)


Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - loss: 3.0244
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 441s 2s/step - loss: 2.1260
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 399s 2s/step - loss: 1.9251
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 471s 2s/step - loss: 1.8052
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 449s 2s/step - loss: 1.7221


In [10]:
import numpy as np
import tensorflow as tf

def generate_text(model, start_string, num_generate=500, temperature=1.0):
    # Convert start string to numbers (vectorize)
    input_eval = [char_to_idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store results
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)   # remove batch dimension

        # Adjust prediction distribution with "temperature"
        predictions = predictions / temperature

        # Sample the next character ID
        predicted_id = tf.random.categorical(tf.expand_dims(predictions[-1], 0), num_samples=1)[0,0].numpy()

        # Add predicted character
        text_generated.append(idx_to_char[predicted_id])

        # Feed predicted char back as next input
        input_eval = tf.expand_dims([predicted_id], 0)

    return start_string + ''.join(text_generated)


In [11]:
print(generate_text(model, start_string="Once upon a time ", num_generate=300, temperature=0.8))


Once upon a time wrea an wherindes ongrour thieve fend ts po B|/>/>Wanangror and. ios this orn a t Bø^. i8ä2Face hale alis t thisple, ade a isioma they st corie tesisices t, mamantor my s t br I )mollioong ber orite B/>R³d f was wavelomy me thear whake re t il r s chmait weralirovo'sthecendenerathe towair a 3 t it 


In [12]:
# Pillow is a Python imaging library.
# It allows us to create, edit, and save images in Python.
!pip install pillow


In [ ]:
!wget -O /content/handwriting.ttf https://github.com/google/fonts/raw/main/ofl/indieflower/IndieFlower-Regular.ttf


In [ ]:
from PIL import Image, ImageDraw, ImageFont

# Example generated text
text = generate_text(model, start_string="Once upon a time ", num_generate=300)

# Create blank white image
img = Image.new("RGB", (1000, 800), color="white")
draw = ImageDraw.Draw(img)

# Load the downloaded font (absolute path!)
font = ImageFont.truetype("/content/handwriting.ttf", 24)

# Write text
draw.multiline_text((10, 10), text, font=font, fill="black", spacing=5)

# Save and show
img.save("generated_handwriting.png")
img.show()
